In [13]:
import os

In [14]:
def write_bsub_file(n, o, l, d, m):    
    f = open("{}/Scripts/{}.bsub".format(d, n), "w")
    f.write('''
#!/bin/bash
### General options
### -- specify queue --   NOTE: TitanX is significantly faster than K80
#BSUB -q gpuv100
#BSUB -gpu "num=1:mode=exclusive_process"
### -- set the job Name --
#BSUB -J s202741-train
### -- ask for number of cores (default: 1) --
#BSUB -n 4
#BSUB -R "span[hosts=1]"
### -- set walltime limit: hh:mm --  maximum 24 hours for GPU-queues right now
#BSUB -W 5:00
# request 5GB of memory
#BSUB -R "rusage[mem=5GB]"
### -- Specify the output and error file. %J is the job-id --
### -- -o and -e mean append, -oo and -eo mean overwrite --
#BSUB -o {}/Logs/{}%J.out
# -- end of LSF options --

# Necessary modules
cd ..
source venv/bin/activate

python trainModelIter4.py 100 \"{}\" "{}" "black_background_500x500.csv" 2 \"{}\" "{}" "load_model_{}"

    '''.format(d, n, o, l, n, d, m))
    f.close()

In [15]:
def create_bsubs(directory):
    script_paths = []
    os.mkdir(directory)
    os.mkdir("{}/Graphs".format(directory))
    os.mkdir("{}/Logs".format(directory))
    os.mkdir("{}/Scripts".format(directory))
    os.mkdir("{}/Models".format(directory))
    optimizers = ["RMSprop"]
    rates = [0.0005, 0.001, 0.002]
    loss_funcs = ["abs_loss_function", "sqrt_abs_min_loss", "smart_sqrt_abs_min_loss"]
    models = ["a","b","c","d","e","f", "g", "h", "i", "j", "k"]
    for model in models:
        for optimizer in optimizers:        
            for rate in rates:
                for loss_func in loss_funcs:
                    rate_s = str(rate).replace('.', '')

                    if optimizer == "Adam":
                        opt = "{}(learning_rate={}, amsgrad=True)".format(optimizer,rate)
                        name = "{}_{}_{}_model_{}_amsgrad_true".format(optimizer, loss_func, rate_s, model)
                        write_bsub_file(name, opt, loss_func, directory, model)

                        opt = "{}(learning_rate={}, amsgrad=False)".format(optimizer,rate)
                        name = "{}_{}_{}_model_{}_amsgrad_false".format(optimizer, loss_func, rate_s)
                        write_bsub_file(name, opt, loss_func, directory, model)

                    else: 
                        opt = "{}(learning_rate={})".format(optimizer,rate)
                        name = "{}_{}_{}_model_{}_".format(optimizer, loss_func, rate_s, model)
                        write_bsub_file(name, opt, loss_func, directory, model)
                    
    return script_paths


In [16]:
def excecute_cmd(cmd):
    out = ""
    os.system('{} > output.txt'.format(cmd))
    file = open('output.txt','r')
    out += file.read()
    file.close()
    os.remove("output.txt")
    return out


def delete_all_jobs():
    job_str = excecute_cmd("qstat gpuv100") 
    jobs = job_str.split(":")
    job_ids = []
    for i in range(1, len(jobs)):
        job_ids.append(jobs[i].replace(" ", "").split("s202741-trai")[1][0:8])
    for job in job_ids:
        os.system('qdel {}'.format(job))
        
def queue_job(path):
    job_str = excecute_cmd("bsub < {}".format(path))
    
def queue_all_jobs(path):
    l = os.listdir("{}/Scripts".format(path))
    for script_file in l:
        queue_job("{}/Scripts/{}".format(path,script_file))
        
def run_jobs(path):
    create_bsubs(path)
    queue_all_jobs(path)

In [17]:
run_jobs("iter7")

In [12]:
delete_all_jobs()



Request <15245013> is being terminated
Request <15245011> is being terminated
Request <15245014> is being terminated
Request <15245012> is being terminated
Request <15245015> is being terminated
Request <15245016> is being terminated
Request <15245017> is being terminated
Request <15245018> is being terminated
Request <15245019> is being terminated
Request <15245020> is being terminated
Request <15245021> is being terminated
Request <15245022> is being terminated
Request <15245023> is being terminated
Request <15245024> is being terminated
Request <15245025> is being terminated
Request <15245026> is being terminated
Request <15245027> is being terminated
Request <15245028> is being terminated
Request <15245029> is being terminated
Request <15245030> is being terminated
Request <15245031> is being terminated
Request <15245032> is being terminated
Request <15245033> is being terminated
Request <15245034> is being terminated
Request <15245035> is being terminated
Request <15245036> is bei

In [ ]:
import os

root_path = '/'
file_path = 'zhome/ab/7/153983/project/scripts'

# Join the root path and file path to create a full path
full_path = os.path.join(root_path, file_path)
os.listdir(full_path)